<h1 align="center"><img align="center" src="https://geoparse.io/graphics/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">All About Points <img src="https://geoparse.io/graphics/point.png" width="10"/> Lines <img src="https://geoparse.io/graphics/line.png" width="40"/> and Polygons <img src="https://geoparse.io/graphics/polygon.png" width="30"/></h3>


#### [HTML](http://geoparse.io/tutorials/karta.html) 
***

# Open Data

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to display:

* Points, lines, and polygons.
* Their heatmaps and clusters.
* Their coverage areas using geospatial cells such as H3, S2, and Geohash.
* Geospatial cells on a map based on their indexes.
* OSM roads and buildings using their IDs.
***

In [ ]:
import warnings

import pandas as pd
import geopandas as gpd

from geoparse.geoparse import plp, SnabbKarta

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

# Postcode

## Shapefile

In [ ]:
bdf = gpd.read_file('../data/os-codepoint-open/boundaries/ESRI/AMD_POSTCODE_AREA_BOUNDARIES.shp').to_crs('epsg:4326')
bdf.head()

In [ ]:
cdf = gpd.read_file('../data/os-codepoint-open/boundaries/ESRI/AMD_POSTCODE_AREA_CENTROIDS.shp').to_crs('epsg:4326')
cdf.head()

In [ ]:
len(bdf), len(cdf)

In [ ]:
SnabbKarta.plp([bdf, cdf])

In [ ]:
cdf.isnull().sum()

## GPKG

In [ ]:
%%time
gdf = gpd.read_file('../data/os-codepoint-open/codepo_gb.gpkg')#.to_crs('epsg:4326')
gdf.head()

In [ ]:
from shapely.geometry import Point

In [ ]:
gdf[gdf.geometry== Point(0, 0)]

In [ ]:
gdf.isnull().sum()

## Parquet

In [ ]:
%%time
gdf = gpd.read_parquet('../data/os-codepoint-open/codepo_gb.parquet')
gdf.head()

In [ ]:
len(gdf), gdf.postcode.nunique(), gdf.geometry.nunique()

In [ ]:
# Count occurrences of each geometry
geometry_counts = gdf.geometry.value_counts()

# Add the count as a new column
gdf['postcode_count'] = gdf.geometry.map(geometry_counts)
gdf.head()

In [ ]:
gdf.postcode_count.value_counts()

In [ ]:
gdf[gdf.postcode_count==4]

In [ ]:
import pandas as pd
import re

def extract_postcode_hierarchy(postcode: str):
    """
    Extract UK postcode hierarchy (Area, District, Sector, Unit) from a full postcode string.
    Returns a tuple: (area, district, sector, unit)
    """
    if not isinstance(postcode, str) or not postcode.strip():
        return (None, None, None, None)

    postcode = postcode.strip().upper().replace(" ", "")
    match = re.match(r"^([A-Z]{1,2})(\d[A-Z\d]?)?(\d)([A-Z]{2})$", postcode)
    if not match:
        return (None, None, None, None)

    area = match.group(1)
    district = f"{match.group(1)}{match.group(2) or ''}"
    sector = f"{district} {match.group(3)}"
    unit = f"{sector}{match.group(4)}"

    return (area, district, sector, unit)

In [ ]:
# Extract hierarchy using list comprehension
results = [extract_postcode_hierarchy(pc) for pc in gdf["postcode"]]
gdf[["postcode_area", "postcode_district", "postcode_sector", "postcode_unit"]] = pd.DataFrame(results, index=gdf.index)
gdf.head()

In [ ]:
SnabbKarta.plp(gdf, point_color='postcode_area')

In [ ]:
from shapely.ops import unary_union
from shapely import concave_hull

In [ ]:
# group by admin_district and compute concave hull for each
pc_area = (
    gdf.groupby("postcode_area")
    .geometry.apply(lambda geom: concave_hull(unary_union(geom), ratio=0.05))
    .reset_index()
)

# turn result into GeoDataFrame
mdf = gpd.GeoDataFrame(pc_area, geometry="geometry", crs=gdf.crs)
#mdf["geometry"] = mdf.geometry.buffer(0.001)
mdf.head()

In [ ]:
SnabbKarta.plp([mdf])#, bdf])

In [ ]:
plp([mdf[mdf.postcode_area=='N'], gdf[gdf.postcode_area=='N'], bdf], point_color='postcode_area')

In [ ]:
gdf[gdf.postcode=='NW8 9ZQ']

In [ ]:
lon, lat, offset = -0.19464, 51.53597, .001

minlon, minlat, maxlon, maxlat = lon-offset, lat-offset, lon+offset, lat+offset
gdf.cx[minlon:maxlon, minlat:maxlat]

In [ ]:
len(gdf)

In [ ]:
gdf.postcode.nunique()

In [ ]:
gdf.geometry.nunique()

In [ ]:
gdf.geometry.value_counts()

In [ ]:
gdf.loc[gdf.country_code.str.startswith('S', na=False), 'country_code'] = 'Scotland'
gdf.loc[gdf.country_code.str.startswith('E', na=False), 'country_code'] = 'England'
gdf.loc[gdf.country_code.str.startswith('W', na=False), 'country_code'] = 'Wales'
gdf.head()

In [ ]:
gdf.country_code.value_counts()

In [ ]:
ddf = pd.read_excel('../data/os-codepoint-open/Codelist.xlsx', sheet_name='CTY', header=None)
ddf.head()

In [ ]:
area_dict = {}
for sheet_name in ['CTY', 'DIS', 'DIW', 'LBO', 'LBW', 'MTD', 'MTW', 'UTA', 'UTE', 'UTW']:
    df = pd.read_excel('../../open-data/data/os-codepoint-open/Codelist.xlsx', sheet_name=sheet_name, header=None)
    for _, row in df.iterrows():
        area_dict[row[1]] = row[0]

In [ ]:
gdf['admin_district_code'] = gdf.admin_district_code.apply(lambda x: area_dict[x] if x in area_dict else x)
gdf['admin_ward_code'] = gdf.admin_ward_code.apply(lambda x: area_dict[x] if x in area_dict else x)
gdf['country_code'] = gdf.country_code.apply(lambda x: area_dict[x] if x in area_dict else x)
gdf.head()

In [ ]:
gdf.isnull().sum()

In [ ]:
# Split into known and missing district codes
kdf = gdf[gdf["admin_district_code"].notna()]     # known df
mdf = gdf[gdf["admin_district_code"].isna()]  # missing df
len(kdf), len(mdf)

In [ ]:
ndf = gpd.sjoin_nearest(   # nearest df
    mdf[['postcode', 'geometry']], 
    kdf[["admin_district_code", 'admin_ward_code', "geometry"]], 
    how="left", 
#    max_distance=None,   # optional: limit to a search radius
 #   distance_col="dist"
)
ndf

In [ ]:
ndf = ndf.drop(columns='index_right').drop_duplicates()
len(ndf)

In [ ]:
gdf.isnull().sum()

In [ ]:
gdf.loc[gdf.admin_district_code.isnull(), "admin_district_code"] = ndf["admin_district_code"]
gdf.loc[gdf.admin_ward_code.isnull(), "admin_ward_code"] = ndf["admin_ward_code"]

In [ ]:
gdf.isnull().sum()

In [ ]:
gdf.to_parquet('../data/os-codepoint-open/codepo_gb_imputed.parquet', index=False)

In [ ]:
%%time
gdf = gpd.read_parquet('../data/os-codepoint-open/codepo_gb_imputed.parquet')
gdf.head()

In [ ]:
SnabbKarta.plp(gdf, point_color='admin_district_code')

In [ ]:
gdf.admin_district_code.nunique()

In [ ]:
tight_polygon = shapely.concave_hull(gdf.geometry.unary_union, ratio=0.1)


In [ ]:
import geopandas as gpd
from shapely.ops import unary_union
from shapely import concave_hull

# group by admin_district and compute concave hull for each
hulls = (
    gdf.groupby("admin_district_code")
    .geometry.apply(lambda geom: concave_hull(unary_union(geom), ratio=0.1))
    .reset_index()
)

# turn result into GeoDataFrame
hulls = gpd.GeoDataFrame(hulls, geometry="geometry", crs=gdf.crs)

hulls


In [ ]:
SnabbKarta.plp(hulls)

# UPRN

In [ ]:
%%time 
gdf = pd.read_parquet('../data/os-open-uprn/osopenuprn_202509.parquet')
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.lon, gdf.lat), crs="EPSG:4326")
gdf = gdf.drop(columns=['lat', 'lon'])
gdf.head()

In [ ]:
len(gdf), gdf.geometry.nunique()

In [ ]:
cdf = gdf.copy()

In [ ]:
gdf = cdf.copy()
len(gdf)

In [ ]:
gdf = gdf.sample(1_000_000).reset_index(drop=True)
gdf.head()

In [ ]:
%%time
if gdf.crs.is_geographic:
    gdf = gdf.to_crs(27700)

In [ ]:
gdf.head()

In [ ]:
%%time
# Define search radius
radius = 100  # meters

# --- Create buffer polygons around each point ---
mdf = gdf.copy()
mdf["geometry"] = gdf.buffer(radius)

# --- Spatial join: find all points within each buffer ---
joined = gpd.sjoin(gdf, mdf, predicate="within", how="left")

# --- Count how many points fall inside each buffer ---
neighbor_counts = joined.groupby("index_right").size() - 1  # exclude self
len(neighbor_counts)

In [ ]:
%%time
# --- Map counts back to original GeoDataFrame ---
gdf["neighbor_count"] = gdf.index.map(neighbor_counts).fillna(0).astype(int)
gdf.head()

In [ ]:
gdf.neighbor_count.describe()

In [ ]:
plp(gdf[gdf.neighbor_count>70].to_crs(4326))

In [ ]:
lat, lon, offset = 51.54512990927041, -0.14470528081862952, .0005

minlon, minlat, maxlon, maxlat = lon-offset, lat-offset, lon+offset, lat+offset
gdf.cx[minlon:maxlon, minlat:maxlat]

In [ ]:
plp(gdf)

In [ ]:
uprn_list = gdf.sample(1000_000).uprn.to_list()

In [ ]:
u = {'geom_type': 'uprn', 'geom_list': uprn_list}

In [ ]:
SnabbKarta.plp(u, aux_gdf=gdf, aux_geom_id='uprn')

In [ ]:
df = pd.DataFrame(u)
del df['geom_type']
df.head()

In [ ]:
SnabbKarta.plp(df, geom_type='uprn', geom_col='geom_list', aux_gdf=gdf, aux_geom_id='uprn')

In [ ]:
SnabbKarta.plp(udf.sample(100_000))

In [ ]:
udf.columns = ['UPRN', 'LAT', 'LON']

In [ ]:
%%time
SnabbKarta.plp(df.sample(10000), 
               geom_col='uprn', 
               geom_type='uprn',
               
               uprn_df=udf, 
               uprn_col='UPRN',
               lat_col='LAT',
               lon_col='LON',
 )

In [ ]:
uprn_list = udf.sample(10000).UPRN.to_list()

In [ ]:
uprn_dict = {'geom_type':'uprn', 'geom_list':uprn_list}

In [ ]:
SnabbKarta.plp(uprn_dict, aux_gdf=udf, aux_geom_id='UPRN')

In [ ]:
if uprn_df is None:
    uprn_df = pd.read_parquet("https://geoparse.io/tutorials/data/osopenuprn_202507.parquet")
    uprn_col, lat_col, lon_col = uprn_df.columns

gdf = gdf.sort_values(by=geom_col).reset_index(drop=True)
gdf = gdf.merge(uprn_df, left_on=geom_col, right_on=uprn_col, how="left")
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf[lon_col], gdf[lat_col]), crs="EPSG:4326")
gdf = gdf.drop(columns=[lat_col, lon_col])


In [ ]:
%%time
SnabbKarta.plp(df.sample(10000), 
               geom_col='uprn', 
               geom_type='uprn',
               
               uprn_df=udf, 
               uprn_col='UPRN',
               lat_col='LAT',
               lon_col='LON',
 )

In [ ]:
%%time
SnabbKarta.plp({'geom_type': 'uprn', 'geom_list': df.uprn.values},
               
               # uprn_df=udf, 
               # uprn_col='UPRN',
               # lat_col='LAT',
               # lon_col='LON',
              )

In [ ]:
df = df.sort_values('uprn').reset_index(drop=True)

In [ ]:
%%time
df = df.merge(udf, on='uprn', how='left')
df.head()

In [ ]:
df.basement.value_counts(normalize=True)

In [ ]:
%%time
df = pd.read_csv("../data/uprn/osopenuprn_202507.csv")
df.head()

In [ ]:
df.head()

In [ ]:
df.columns = ['uprn', 'lat', 'lon']

In [ ]:
df.isnull().sum()

In [ ]:
len(df)

In [ ]:
df.uprn.nunique()

In [ ]:
df.to_parquet('../data/uprn/osopenuprn_202507.parquet', index=False)

# USRN

In [ ]:
gdf = gpd.read_parquet('../../open-data/data/os-open-roads/road_link.parquet', bbox=(-0.5,51.3, 0.5,52))
len(gdf)

In [ ]:
gdf.head(2)

In [ ]:
SnabbKarta.plp(gdf.sample(100_000))

In [ ]:
%%time
gdf = gpd.read_parquet('/Users/abbas/repo/open-data/data/os-open-usrn/osopenusrn_202509.parquet', bbox=(-0.5,51.25, 0.5,51.75))
#gdf = gpd.read_file('~/Dropbox/notebook/data/fetch/polygons/uk_rgn.gpkg')
len(gdf)

In [ ]:
u = {'geom_type': 'usrn', 'geom_list': gdf.usrn.to_list()}

In [ ]:
SnabbKarta.plp(u, aux_gdf=gdf, aux_geom_id='usrn')

In [ ]:
df = pd.DataFrame(u)
del df['geom_type']
df.head()

In [ ]:
SnabbKarta.plp(df, geom_type='usrn', geom_col='geom_list', 
               aux_gdf=gdf, aux_geom_id='usrn')

In [ ]:
gdf.head()

In [ ]:
len(gdf)

In [ ]:
import shapely

In [ ]:
type(gdf.geometry.type.unique()[0])

In [ ]:
for g in gdf.geometry.type.unique():
    print(len(gdf[gdf.geometry.type==g]))

In [ ]:
df = pd.read_csv('/Users/abbas/Dropbox/notebook/data/fetch/macrodb/postcode_doogal/sectors_10100.csv.zip')
df['geometry'] = [shapely.wkb.loads(item, hex=True) for item in df.geom]

gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf = gdf[['name', 'geometry']]
gdf.columns = ['pc_sector', 'geometry']
gdf.head()

In [ ]:
SnabbKarta.plp(gdf)

In [ ]:
gdf.geometry.type.value_counts()

In [ ]:
reg_list = ['North West', 'London']

In [ ]:
reg_dict = {'geom_type': 'postcode', 'geom_list': reg_list}

In [ ]:
SnabbKarta.plp(reg_dict, 
               aux_gdf=gdf, aux_geom_id='name'
              )

In [ ]:
pc_list = gdf.sample(1000).pc_sector.to_list()
pc_dict = {'geom_type':'postcode', 'geom_list': pc_list}

In [ ]:
SnabbKarta.plp(pc_dict, 
               aux_gdf=gdf, aux_geom_id='pc_sector'
              )

In [ ]:
usrn_list = gdf.sample(100000).usrn.to_list()

In [ ]:
usrn_dict = {'geom_type':'usrn', 'geom_list': usrn_list}

In [ ]:
SnabbKarta.plp(usrn_dict, 
               aux_gdf=gdf, aux_geom_id='usrn'
              )

In [ ]:
df = pd.DataFrame({'street_id': usrn_list})
df.head()

In [ ]:
SnabbKarta.plp(df, geom_type='usrn', geom_col='street_id',
               aux_gdf=gdf, aux_geom_id='usrn'
              )

In [ ]:
gdf

In [ ]:
%%time
gdf = gpd.read_file('../data/usrn/osopenusrn_202509_processed.gpkg')
len(gdf)

In [ ]:
gdf.head()

In [ ]:
cdf = gdf.copy()

In [ ]:
from geoparse import GeomUtils

In [ ]:
gdf = cdf.copy()
gdf = gdf.sample(10000).reset_index(drop=True)

In [ ]:
%%time
gdf['geometry'] = GeomUtils.flatten_3d(gdf.geometry)
gdf.head()

In [ ]:
%%time
gdf = gdf.to_crs('epsg:4326')
gdf.head()

In [ ]:
gdf.geometry.geom_type.value_counts()

In [ ]:
gdf.street_type.value_counts(normalize=True)

In [ ]:
plp(gdf[gdf.geometry.geom_type == 'LineString'].sample(10000))

In [ ]:
test_flatten_3d_all_types()

# Boundaries

In [ ]:
gdf = gpd.read_parquet('../data/ons-output-area/msoa.parquet')
gdf.head()

In [ ]:
gdf.describe()

In [ ]:
gdf = gpd.read_parquet('../data/ons-output-area/lsoa.parquet')
gdf.head()

In [ ]:
gdf = gpd.read_parquet('../data/ons-output-area/oa.parquet')
gdf.head()

In [ ]:
SnabbKarta.plp(gdf)

# Income

In [ ]:
df = pd.read_parquet('../data/ons-income/net_income_after_housing_costs.parquet')
df.head()

In [ ]:
df.info()

In [ ]:
len(df)

In [ ]:
df.isnull().sum()

In [ ]:
df = pd.read_excel('../data/ons-income/ons-income.xlsx', sheet_name='Total annual income', skiprows=4)
df.head()

In [ ]:
len(df), df['MSOA code'].nunique()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
# Sort by mean income
df_sorted = df.sort_values("Total annual income (£)").reset_index(drop=True)

# Plot scatter
plt.figure(figsize=(8, 5))

plt.scatter(df_sorted.index, df_sorted["Upper confidence limit (£)"], label="Upper limit", marker="o", s=1)
plt.scatter(df_sorted.index, df_sorted["Total annual income (£)"], label="Total annual income (£)", marker="o", s=1)
plt.scatter(df_sorted.index, df_sorted["Lower confidence limit (£)"], label="Lower limit", marker="o", s=1)

# Annotate x-axis with MSOA names (optional, but may be dense)
#plt.xticks(df_sorted.index, df_sorted["MSOA name"], rotation=45, ha='right')

plt.title("Total Annual Household Income with Confidence Intervals")
plt.xlabel("MSOA (sorted by total annual income)")
plt.ylabel("Annual Household Income (£)")
plt.legend()
plt.tight_layout()
plt.show()

# Income Choropleth

In [ ]:
df = pd.read_excel('../data/ons-income/ons-income.xlsx', sheet_name='Total annual income', skiprows=4)
df.head(2)

In [ ]:
gdf.head(2)

In [ ]:
from geoparse.geoparse import Karta

In [ ]:
gdf = pd.merge(gdf[['msoa11cd', 'geometry']],
         df[['MSOA code', 'MSOA name', 'Total annual income (£)']],
         left_on='msoa11cd', 
         right_on='MSOA code',
         how='left'
        ).drop(columns='MSOA code')
gdf.head()

In [ ]:
k = Karta.choropleth(gdf,
                 columns=['MSOA name', 'Total annual income (£)'],
                 legend='Total Annual Household Income (£)',
                 bins=[20000, 35000, 50000, 75000, 110000]
                )

In [ ]:
k.save('/Users/abbas/Desktop/total_income.html')